# Power manifold results

## Setup

In [1]:
import sys
import numpy as np
from sklearn.metrics import roc_auc_score


## Load results

In [2]:
show_all = True
n_runs = 3

In [3]:
algo_filenames = []
algo_additionals = []
algo_labels = []
algo_dividers = []

def add_algo(filename, add, label, shortlist=False):
    if show_all or shortlist:
        algo_filenames.append(filename)
        algo_additionals.append(add)
        algo_labels.append(label)
    
    
def add_divider():
    algo_dividers.append(len(algo_filenames))
    

add_algo("flow", "_small", "EF")
add_algo("flow", "_small_long", "EF (long)", True)
add_algo("flow", "_small_shallow_long", "EF (shallow, long)")

add_divider()
add_algo("pie", "_small", "PIE") 
add_algo("pie", "_small_pieepsilon01", "PIE (eps = 0.1)") 
add_algo("pie", "_small_pieepsilon003", "PIE (eps = 0.03)") 
add_algo("pie", "_small_pieepsilon0003", "PIE (eps = 0.003)") 
add_algo("pie", "_small_pieepsilon0001", "PIE (eps = 0.001)") 
add_algo("pie", "_small_pieepsilon00003", "PIE (eps = 0.0003)") 
add_algo("pie", "_small_pieepsilon00001", "PIE (eps = 0.0001)") 
add_algo("pie", "_small_long", "PIE (long)", True) 
add_algo("pie", "_small_shallow_long", "PIE (shallow, long)") 

add_divider()
add_algo("mf", "_small", "MLF-S")
add_algo("mf", "_small_morenll", "MLF-S (large lambda)")
add_algo("mf", "_small_long", "MLF-S (long)", True)
add_algo("mf", "_small_shallow_long", "MLF-S (shallow, long)")

add_divider()
add_algo("mf", "_small_alternate", "MLF-A")
add_algo("mf", "_small_alternate_wdecay", "MLF-A (weight decay)")
add_algo("mf", "_small_alternate_long", "MLF-A (long)", True)
add_algo("mf", "_small_alternate_wdecay_long", "MLF-A (long, weight decay)")
add_algo("mf", "_small_alternate_shallow_long", "MLF-A (shallow, long)")
add_algo("mf", "_small_alternate_wdecay_shallow_long", "MLF-A (shallow, long, weight decay)")

add_divider()
add_algo("gamf", "_small_largebs", "MLF-OT")
add_algo("gamf", "_small_wdecay_largebs", "MLF-OT (weight decay)")
add_algo("gamf", "_small_hugebs", "MLF-OT (5k bs)") 
add_algo("gamf", "_small_wdecay_hugebs", "MLF-OT (5k bs, weight decay)") 
add_algo("gamf", "_small_largebs_long", "MLF-OT (long)", True) 
add_algo("gamf", "_small_wdecay_largebs_long", "MLF-OT (long, weight decay)") 
add_algo("gamf", "_small_largebs_shallow_long", "MLF-OT (shallow, long)") 
add_algo("gamf", "_small_wdecay_largebs_shallow_long", "MLF-OT (shallow, long, weight decay)") 

add_divider()
add_algo("gamf", "_small_largebs", "MLF-OTA")
add_algo("gamf", "_small_wdecay_largebs", "MLF-OTA (weight decay)")
add_algo("gamf", "_small_largebs_long", "MLF-OTA (long)", True) 
add_algo("gamf", "_small_wdecay_largebs_long", "MLF-OTA (long, weight decay)") 
add_algo("gamf", "_small_largebs_shallow_long", "MLF-OTA (shallow, long)") 
add_algo("gamf", "_small_wdecay_largebs_shallow_long", "MLF-OTA (shallow, long, weight decay)") 

add_divider()
add_algo("emf", "_small", "EMLF-S")
add_algo("mef", "_small_morenll", "EMLF-S (large lambda)")
add_algo("emf", "_small_long", "EMLF-S (long)", True)
add_algo("emf", "_small_shallow_long", "EMLF-S (shallow, long)")

add_divider()
add_algo("emf", "_small_alternate", "EMLF-A")
add_algo("emf", "_small_alternate_wdecay", "EMLF-A (weight decay)")
add_algo("emf", "_small_alternate_long", "EMLF-A (long)", True)
add_algo("emf", "_small_alternate_wdecay_long", "EMLF-A (long, weight decay)")
add_algo("emf", "_small_alternate_shallow_long", "EMLF-A (shallow, long)")
add_algo("emf", "_small_alternate_wdecay_shallow_long", "EMLF-A (shallow, long, weight decay)")


In [4]:
def load(name, shape, numpyfy=True, result_dir="../data/results"):
    all_results = []
    
    for algo_filename, algo_add in zip(algo_filenames, algo_additionals):
        algo_results = []
            
        for run in range(n_runs):
            run_str = "" if run == 0 else "_run{}".format(run)
            try:
                this_result = np.load(
                    "{}/{}_2_power{}{}_{}.npy".format(
                        result_dir, algo_filename, algo_add, run_str, name
                    )
                )
                if (not numpyfy) or (shape is None) or np.product(this_result.shape) == np.product(shape):
                    algo_results.append(this_result.reshape(shape))
                else:
                    print(this_result.shape, shape)
                    algo_results.append(np.nan*np.ones(shape))
                    
            except FileNotFoundError as e:
                if shape is None:
                    algo_results.append(None)
                else:
                    algo_results.append(np.nan*np.ones(shape))
            
        all_results.append(algo_results)
    
    if numpyfy:
        try:
            all_results = np.array(all_results, dtype=np.float)
            print(all_results.shape)
        except Exception as e:
            print(name)
            print(len(all_results))
            print([len(res) for res in all_results])
            print([[len(r) for r in res] for res in all_results])
            raise
    else:
        print(len(all_results))
        
    return all_results


model_gen_x = load("samples", None, numpyfy=False)
model_gen_logp = load("samples_likelihood", (10000,))
model_gen_distance = load("samples_manifold_distance", (10000,))
model_test_logp = load("model_log_likelihood_test", (101, 1000,))
model_test_reco_error = load("model_reco_error_test", (1000,))
model_ood_logp = load("model_log_likelihood_ood", (101, 1000,))
model_ood_reco_error = load("model_reco_error_ood", (1000,))
model_posterior_samples = load("posterior_samples", (5000, 1,))
model_mmds = load("mmd", (1,))


46
(46, 3, 10000)
(46, 3, 10000)
(46, 3, 101, 1000)
(46, 3, 1000)
(46, 3, 101, 1000)
(46, 3, 1000)
(1000, 1) (5000, 1)
(1000, 1) (5000, 1)
(1000, 1) (5000, 1)
(46, 3, 5000, 1)
(46, 3, 1)


In [5]:
model_gen_logp.shape

(46, 3, 10000)

In [6]:
def remove_nans(*results):
    passes = all([np.all(np.isfinite(result)) for result in results])
    
    if passes:
        return results
    else:
        return [np.nan * np.ones_like(result) for result in results]


def remove_nans_from_lists(*raws):
    # raws[quantity][algo]
    n_quantities = len(raws)
    n_algos = len(raws[0])
    
    for raw in raws:
        assert len(raw) == n_algos
    
    cleans = [[[] for _ in range(n_algos)] for _ in range(n_quantities)]
    
    for i in range(n_algos):
        for k in range(n_runs):
            clean = remove_nans(*[raw[i][k] for raw in raws])
            for j in range(n_quantities):
                cleans[j][i].append(clean[j])
            
    cleans = [np.array(clean) for clean in cleans]
    
    # cleans[quantity][algo]
    return cleans
            
raw = [model_gen_logp, model_gen_distance, model_test_logp, model_ood_logp, model_ood_reco_error, model_posterior_samples, model_mmds]
clean = remove_nans_from_lists(*raw)
model_gen_logp, model_gen_distance, model_test_logp, model_ood_logp, model_ood_reco_error, model_posterior_samples, model_mmds = clean
           

In [7]:
model_gen_logp.shape

(46, 3, 10000)

In [8]:
def load_truth(name, samples=True):
    if samples:
        return np.asarray([
            np.load("../data/samples/power/{}{}.npy".format(
                name, run_str
            ))
            for run_str in [""] + ["_run{}".format(i) for i in range(1, n_runs)]
        ])
    else:
        return np.asarray([
            np.load("../data/results/truth_power{}_{}.npy".format(
                run_str, name
            ))
            for run_str in [""] + ["_run{}".format(i) for i in range(1, n_runs)]
        ])

test_x = load_truth("x_test", True)
test_distance = np.zeros((test_x.shape[0], 1))
test_logp = load_truth("true_log_likelihood_test", False)
true_posterior_samples = load_truth("posterior_samples", False)

param_grid = np.linspace(-1, 1, 101)


## Calculate metrics

In [9]:
n_observed = 20
min_logp = -100.
max_distance = 10.

model_gen_mean_logp = np.mean(np.clip(model_gen_logp, min_logp, None), axis=2)
model_gen_mean_distance = np.mean(np.clip(model_gen_distance, None, max_distance), axis=2)
model_observed_nll = -2. * np.sum(model_test_logp[:,:,:,:n_observed], axis=-1)

test_mean_logp = np.mean(np.clip(test_logp, min_logp, None), axis=1)
test_mean_distance = np.mean(np.clip(test_distance, None, max_distance), axis=1)
true_observed_nll = -2. * np.sum(test_logp[:,:,:n_observed], axis=-1)


In [10]:
def calculate_roc_auc(x0, x1):
    assert x0.shape == x1.shape
    old_shape = x0.shape[:-1]
    x0 = x0.reshape(-1, x0.shape[-1])
    x1 = x1.reshape(-1, x1.shape[-1])
    
    aucs = []
    for x0_, x1_ in zip(x0, x1):
        if not np.all(np.isfinite(np.hstack((x0_, x1_)))):
            aucs.append(np.nan)
            continue
            
        auc = roc_auc_score(
            np.hstack((np.zeros(x0_.shape[0], dtype=np.int), np.ones(x1_.shape[0], dtype=np.int))),
            np.hstack((x0_, x1_)),
        )
        auc_flipped = roc_auc_score(
            np.hstack((np.zeros(x0_.shape[0], dtype=np.int), np.ones(x1_.shape[0], dtype=np.int))),
            - np.hstack((x0_, x1_)),
        )
        aucs.append(max(auc, auc_flipped))
        
    aucs = np.asarray(aucs)
    aucs = aucs.reshape(old_shape)
    return aucs


model_auc_logp = calculate_roc_auc(model_test_logp[:,:,50,:], model_ood_logp[:,:,50,:])
model_auc_err = calculate_roc_auc(model_test_reco_error, model_ood_reco_error)
model_auc_use_err = (model_auc_err > model_auc_logp)
model_auc = np.maximum(model_auc_err, model_auc_logp)


/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in greater


## Best metrics

In [11]:
best_mmd, best_dist, best_auc = 0, 0, 0

In [12]:
best_mmd = np.nanargmin(np.mean(model_mmds, axis=1))
print(algo_labels[best_mmd])

best_dist = np.nanargmin(np.mean(model_gen_mean_distance, axis=1))
print(algo_labels[best_dist])

best_auc = np.nanargmax(np.mean(model_auc, axis=1))
print(algo_labels[best_dist])

MLF-S
EF (long)
EF (long)


## Print metrics

In [13]:
def print_results(
    include_err=False, include_n_runs=False, l_label=max([len(l) for l in algo_labels]), l_mean=6, l_err=5
):
    # How to format the numbers
    l_result = l_mean + int(include_err) * (3 + l_err) + int(include_n_runs) * 4
    l_total = l_label + 1 + 3 * (3 + l_result)
    
    def _f(val):
        if not np.any(np.isfinite(val)):
            return " "*l_result
        
        result = "{:>{}.{}f}".format(np.nanmean(val), l_mean, l_mean - 2)
        if include_err:
            result += " ({:>{}.{}f})".format(np.nanstd(val), l_err, l_err - 2)
        if include_n_runs:
            result += " [{:1n}]".format(np.sum(np.isfinite(val)))
        return result
        
    
    # Header
    print(
        "{2:<{0}.{0}s} | {3:>{1}.{1}s} | {4:>{1}.{1}s} | {5:>{1}.{1}s} ".format(
            l_label, l_result, "", "Mean dist", "OOD AUC", "Post. MMD"
        )
    )
    print("-"*l_total)

    # Iterate over methods
    for i, (label, dist, mmd, auc) in enumerate(zip(algo_labels, model_gen_mean_distance, model_mmds, model_auc)):
        # Divider
        if i in algo_dividers and show_all:
            print("-"*l_total)
            
        # Print results
        print("{1:<{0}.{0}s} | {2}{5}| {3}{6}| {4}{7}".format(
            l_label, label,
            _f(dist), _f(auc),  _f(mmd),
            "*" if i == best_dist else " ", "*" if i == best_auc else " ", "*" if i == best_mmd else " "
        ))


In [14]:
print_results(include_n_runs=True)

                                      |  Mean dist |    OOD AUC |  Post. MMD 
-----------------------------------------------------------------------------
EF                                    | 0.0239 [3] | 0.8857 [3] | 0.1439 [3] 
EF (long)                             | 0.0075 [3]*| 0.9360 [3]*| 0.1125 [3] 
EF (shallow, long)                    | 0.0121 [3] | 0.9195 [3] | 0.1303 [3] 
-----------------------------------------------------------------------------
PIE                                   | 0.0371 [3] | 0.8341 [3] | 0.1770 [3] 
PIE (eps = 0.1)                       | 0.0215 [3] | 0.8657 [3] | 0.1687 [3] 
PIE (eps = 0.03)                      | 0.0124 [3] | 0.8611 [3] | 0.0662 [3] 
PIE (eps = 0.003)                     | 0.0166 [3] | 0.8841 [3] | 0.2027 [3] 
PIE (eps = 0.001)                     | 0.0837 [3] | 0.7794 [3] | 0.2861 [3] 
PIE (eps = 0.0003)                    | 0.0145 [3] | 0.8647 [3] | 0.1275 [3] 
PIE (eps = 0.0001)                    | 0.0218 [3] | 0.8916 [3] 

In [15]:
print_results(include_err=True)

                                      |      Mean dist |        OOD AUC |      Post. MMD 
-----------------------------------------------------------------------------------------
EF                                    | 0.0239 (0.020) | 0.8857 (0.031) | 0.1439 (0.154) 
EF (long)                             | 0.0075 (0.003)*| 0.9360 (0.007)*| 0.1125 (0.058) 
EF (shallow, long)                    | 0.0121 (0.002) | 0.9195 (0.017) | 0.1303 (0.072) 
-----------------------------------------------------------------------------------------
PIE                                   | 0.0371 (0.022) | 0.8341 (0.027) | 0.1770 (0.144) 
PIE (eps = 0.1)                       | 0.0215 (0.009) | 0.8657 (0.035) | 0.1687 (0.186) 
PIE (eps = 0.03)                      | 0.0124 (0.007) | 0.8611 (0.037) | 0.0662 (0.038) 
PIE (eps = 0.003)                     | 0.0166 (0.009) | 0.8841 (0.037) | 0.2027 (0.114) 
PIE (eps = 0.001)                     | 0.0837 (0.029) | 0.7794 (0.046) | 0.2861 (0.161) 
PIE (eps =